In [1]:
!pip install tensorflow
!pip install prophet
!pip install xgboost
!pip install h5py
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 3.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 42.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 12.5 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 27.4 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 69.2 MB/s eta 0:00:00:00:01


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 118.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 7.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 26.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.6 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 16.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 5.0 MB/s eta 0:00:00:00:01


In [2]:
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
import os
import glob
import boto3
import s3fs
import tempfile
import joblib
import h5py
from config_module import Config
config = Config()
# **IMPORTANT**: Have to do this line *before* importing tensorflow
os.environ['PYTHONHASHSEED'] = str(2)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import tensorflow.keras as keras
import tarfile
tf.get_logger().setLevel('ERROR')
tf.keras.config.disable_interactive_logging()
from tensorflow.keras.models import save_model, model_from_json, Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from pickle import dump, load
from keras.models import load_model
import warnings

import random
def reset_random_seeds():
    os.environ['PYTHONHASHSEED'] = str(2)
    tf.random.set_seed(2)
    np.random.seed(2)
    random.seed(2)
    tf.keras.utils.set_random_seed(2024)

# Make some random data
reset_random_seeds()

# Set seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
pd.options.mode.chained_assignment = None

# Import modules
import itertools
import logging
import math
import pickle
from datetime import datetime

# import ast
import seaborn as sn
# import xgboost as xgb
from matplotlib import pyplot as plt
from pandas.plotting import register_matplotlib_converters
# from prophet import Prophet
from sklearn.feature_selection import SelectKBest, f_regression
# from sklearn.linear_model import Lasso, LinearRegression, Ridge
# from sklearn.metrics import (mean_absolute_error, mean_absolute_percentage_error,
#                              mean_squared_error)
# from sklearn.model_selection import (GridSearchCV, TimeSeriesSplit,
#                                      train_test_split)
# from sklearn.preprocessing import StandardScaler
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# from statsmodels.tsa.arima.model import ARIMA
# from statsmodels.tsa.holtwinters import ExponentialSmoothing
# from statsmodels.tsa.seasonal import seasonal_decompose
# from statsmodels.tsa.stattools import adfuller
# from statsmodels.tsa.statespace.sarimax import SARIMAX
# import statsmodels.api as sm
warnings.filterwarnings('ignore')
random.seed(12)

2024-09-15 09:31:36.035568: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-15 09:31:36.055866: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-15 09:31:36.061943: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-15 09:31:36.076260: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-15 09:31:37.490576: W tensorflow/comp

In [3]:
start_time = datetime.now()

In [4]:
def read_file(file_path):
    file_extension = os.path.splitext(file_path)[1].lower()
    
    if file_extension == '.xlsx':
        # Read Excel file
        data = pd.read_excel(file_path)
    elif file_extension == '.csv':
        # Read CSV file
        data = pd.read_csv(file_path)
    elif file_extension == '.txt':
        # Read TXT file
        data = pd.read_csv(file_path)
    else:
        raise ValueError("Unsupported file format: {}".format(file_extension))
    
    return data

In [5]:
def get_latest_file(folder_path, file_prefix):
    # Construct the search pattern for the CSV files with the given prefix
    search_pattern = os.path.join(folder_path, f"{file_prefix}*")
    
    
    # Get the list of matching files
    csv_files = glob.glob(search_pattern)
    
    
    if not csv_files:
        print("No CSV files found.")
        return None
    
    # Extract the datetime part and convert it to a datetime object
    csv_files_with_dates = []
    for file in csv_files:
        try:
            # Extract the datetime stamp from the file name
            datetime_stamp = file[len(folder_path) + len(file_prefix) + 1:-5]
            file_datetime = datetime.strptime(datetime_stamp, "%y%m%d_%H%M%S")
            csv_files_with_dates.append((file, file_datetime))
        except ValueError:
            print(f"Could not parse date from file name: {file}")
    
    if not csv_files_with_dates:
        print("No valid CSV files found.")
        return None
    
    # Sort the files by datetime in descending order and get the latest one
    latest_file = max(csv_files_with_dates, key=lambda x: x[1])[0]
    
    return latest_file

In [6]:
def get_latest_s3_file(bucket_name, path_prefix, file_prefix):
    s3 = boto3.client('s3')
    
    # List all objects under the given path_prefix
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=path_prefix)
    
    if 'Contents' not in response:
        print("No files found.")
        return None
    
    # Filter objects that start with the specific file path_prefix
    filtered_files = [
        obj for obj in response['Contents'] 
        if obj['Key'].startswith(f"{path_prefix}{file_prefix}")
    ]
    
    if not filtered_files:
        print("No files found with the specific path_prefix.")
        return None
    
    # Extract the date from each file name and store along with the key
    files_with_dates = []
    for obj in filtered_files:
        key = obj['Key']
        # Assuming the date is at the end of the filename after the last underscore
        try:
            # Get the base name without directory path
            base_name = os.path.basename(key)
            name_without_extension = os.path.splitext(base_name)[0]
            date_str = name_without_extension.split('_')[-1]  # Get the date string part
            file_date = datetime.strptime(date_str, "%Y-%m-%d")  # Parse date
            files_with_dates.append((key, file_date))
        except ValueError as e:
            print(f"Error parsing date from file name {key}: {e}")

    if not files_with_dates:
        print("No valid files with dates found.")
        return None

    # Find the file with the latest date
    latest_file_key, latest_date = max(files_with_dates, key=lambda x: x[1])
    
    return latest_file_key

AWS Credentials

In [7]:
# Specify your S3 bucket and path
bucket = config.get_value('bucket')
prefix_output_file = config.get_value('prefix_cleaned_training_file_path')
file_name_prefix = config.get_value('cleaned_training_filename_prefix')
file_path = get_latest_s3_file(bucket,prefix_output_file,file_name_prefix)

old_data = pd.read_excel('s3://'+f'{bucket}/{file_path}')
parts_list = old_data['part_number'].unique().tolist()

prefix_lstm_pickle = f"{config.get_value('primary_path')}/{config.get_value('tpcap_model')}/{config.get_value('model_training_date')}/{'lstm'}"
prefix_output = f"{config.get_value('primary_path')}/{config.get_value('tpcap_model')}/{config.get_value('model_training_date')}/{config.get_value('model_output_folder_name')}"

# lstm_pickle_path = f's3://{bucket}/{prefix_lstm_pickle}'
# lstm_predict_data_path = f's3://{bucket}/{prefix_lstm_train_data_path}'

In [8]:
print(file_path)

persist-bo/persist-bo-demand-forecast/cleaned-training-data/clean_data_imputed_2024-09-13.xlsx


In [9]:
len(parts_list)

6525

In [10]:
def clean_subset_data(data_new):
    """
    Clean and subset input data for time series analysis.

    Args:
    - data_new (DataFrame): Input DataFrame containing raw monthly data.

    Returns:
    - DataFrame: Cleaned and subsetted DataFrame containing part number, period (month-year), and total quantity per month.
    """
    # Convert 'period' column to datetime format
    data_new['period'] = pd.to_datetime(data_new['period'])
    
    # Extract month-year format from datetime and store as string
    data_new['period'] = data_new['period'].dt.strftime('%Y-%m')
    
    # Sort data by 'part_number' and 'period'
    data_new = data_new.sort_values(['part_number', 'period'])
    
    # Filter data to include periods from January 2021 onwards
    data_new = data_new[data_new['period'] >= '2021-01']
    
    # Select relevant columns: 'part_number', 'period', 'Total_qty_per_month'
    data_new = data_new[['part_number', 'period', 'Total_qty_per_month']]
    
    return data_new

data_new_1 = clean_subset_data(old_data)

In [11]:
def create_trainable_dataset(dataframe, n_inputs, n_outputs):
    """
    Create a trainable dataset for sequence prediction tasks.

    Args:
    - dataframe (DataFrame): Input DataFrame containing sequential data.
    - n_inputs (int): Number of time steps to use as input (X).
    - n_outputs (int): Number of time steps to predict as output (Y).

    Returns:
    - tuple: Tuple containing numpy arrays X and Y, where:
        - X: Array of input sequences, each of shape (n_inputs, n_features).
        - Y: Array of output sequences, each of shape (n_outputs, n_features).
    """
    X, Y = list(), list()
    
    # Iterate over the dataframe to create input-output pairs
    for i in range(len(dataframe) - n_inputs - n_outputs + 1):
        # Slice the dataframe to extract X (input sequence) and Y (output sequence)
        X.append(dataframe.iloc[i:(i + n_inputs), :].values)
        Y.append(dataframe.iloc[(i + n_inputs):(i + n_inputs + n_outputs), :].values)
    
    # Convert lists to numpy arrays
    X = np.array(X)
    Y = np.array(Y)
    
    return X, Y

def scale_data(train_data, test_data):
    """
    Scale the 'Total_qty_per_month' column in both training and test datasets using MinMaxScaler.

    Args:
    - train_data (DataFrame): Training dataset containing 'Total_qty_per_month' column to be scaled.
    - test_data (DataFrame): Test dataset containing 'Total_qty_per_month' column to be scaled.

    Returns:
    - tuple: Tuple containing:
        - t_transformer (MinMaxScaler): Scaler object fitted on training data.
        - train_data_scaled (DataFrame): Training dataset with 'Total_qty_per_month' column scaled.
        - test_data_scaled (DataFrame): Test dataset with 'Total_qty_per_month' column scaled.
    """
    # Initialize MinMaxScaler
    t_transformer = MinMaxScaler()

    # Fit and transform on training data
    t_transformer = t_transformer.fit(train_data[['Total_qty_per_month']])
    train_data['Total_qty_per_month'] = t_transformer.transform(train_data[['Total_qty_per_month']])

    # Transform test data
    if test_data.shape[0]>0:
        test_data['Total_qty_per_month'] = t_transformer.transform(test_data[['Total_qty_per_month']])
    else:
        test_data=pd.DataFrame()

    return t_transformer, train_data, test_data

In [12]:
n_input=12
n_output=1 #1 day 24x1=24 hours

forecast_window_val = 3

In [13]:
def create_model_output(data_lag1, training_end_date_val, inp_n, out_n, s3_bucketname, model_pickle_folder_prefix, forecast_window_val, prod_list):
    """
    Train LSTM models for each product and generate forecasts.

    Args:
    - data_lag1 (DataFrame): DataFrame containing lagged data for lag 1.
    - training_end_date_val (str): Training end date in 'YYYY-MM' format.
    - inp_n (int): Number of input time steps for LSTM model.
    - out_n (int): Number of output time steps for LSTM model.
    - model_pickle_folder_url (str): Folder path to save/load LSTM models.
    - model_folder_url (str): Folder path to save final file
    - forecast_window_val (int): Number of future periods to forecast.
    - prod_list (list): List of products to run the forecast.
    - filename_lstm (str): basefile name for LSTM final data

    Returns:
    - None: Saves forecast results for each product and lag to Excel files.
    """
    out_data_1 = pd.DataFrame()
    current_datetime = datetime.now().strftime("%y%m%d_%H%M%S")
    # Initialize the S3 filesystem
    fs = s3fs.S3FileSystem()
    s3_client = boto3.client('s3')
    for i, product in enumerate(prod_list):
        fin_data_1 = pd.DataFrame()
        for t in range(1):  # Loop through lag 1, lag 2, and lag 3
            try:
                print(f'Running for product number {i+1}')
                # Select appropriate data based on lag
                if t == 0:
                    data_new = data_lag1[:]
                
                # Filter data for the current product and time period
                all_data = data_new[data_new['part_number'] == product]
                all_data['period'] = pd.to_datetime(all_data['period'])
                all_data = all_data[all_data['period'] <= pd.to_datetime(training_end_date_val) + relativedelta(months=t+3)]
    
                train_data = all_data[all_data['period'] <= pd.to_datetime(training_end_date_val) + relativedelta(months=t)]
                test_data = all_data[(all_data['period'] > pd.to_datetime(training_end_date_val) + relativedelta(months=t)) &
                                     (all_data['period'] <= pd.to_datetime(training_end_date_val) + relativedelta(months=t+3))]
    
                # Ensure columns are consistent between train and test data
                train_data = train_data[train_data.columns.intersection(test_data.columns)]
    
                # Combine train and test data for scaling
                all_data = pd.concat([train_data, test_data], ignore_index=True)
    
                # Scale data
                train_data.set_index('period', inplace=True)
                train_data = train_data[['Total_qty_per_month']]
                test_data.set_index('period', inplace=True)
                test_data = test_data[['Total_qty_per_month']]
                all_data.set_index('period', inplace=True)
    
                t_transformer, s_train_data, s_test_data = scale_data(train_data, test_data)
    
                # Create input-output pairs for LSTM model
                X_train, Y_train = create_trainable_dataset(s_train_data, inp_n, out_n)
    
                # Build or load LSTM model
                if t == 0:
                    model = Sequential()
                    model.add(LSTM(units=50, return_sequences=True, input_shape=X_train.shape[1:]))
                    model.add(LSTM(units=50))
                    model.add(Dense(units=Y_train.shape[1]))
                    model.compile(loss='mean_squared_error', optimizer='adam')
                    model.fit(X_train, Y_train, epochs=100, batch_size=8, verbose=0)                    
                    model.save(f"/tmp/lstm-n-{product}.h5")  # creates a HDF5 file 'my_model.h5'
                    s3_client.upload_file(Filename=f"/tmp/lstm-n-{product}.h5",
                                      Bucket=s3_bucketname,
                                      Key=f"{model_pickle_folder_prefix}/lstm-n-{product}.h5")
                    # Delete the local file after upload
                    os.remove(f"/tmp/lstm-n-{product}.h5")
                # Function to perform multi-step forecasts
                def multi_step_forecasts_future(n_past, n_future):
                    x_past = X_train[-n_past-1:, :, :][:1]  # Last observed input sequence
                    y_past = Y_train[-n_past-1]             # Last observed target value
                    y_future = []                            # Predicted target values
    
                    for _ in range(n_past + n_future):
                        x_past = np.append(x_past[:, 1:, :], y_past.reshape(1, 1, 1), axis=1)
                        y_past = model.predict(x_past)
                        y_future.append(y_past.flatten()[0])
    
                    # Transform forecasts back to original scale
                    y_future = t_transformer.inverse_transform(np.array(y_future).reshape(-1, 1)).flatten()
    
                    # Create DataFrame for past and future forecasts
                    df_past = all_data.rename(columns={'Total_qty_per_month': 'Actual'}).copy()
                    df_future = pd.DataFrame(
                        index=pd.date_range(start=train_data.index[-n_past-1] + relativedelta(months=1),
                                            periods=n_past + n_future, freq='MS'),
                        columns=['Forecast'],
                        data=y_future
                    )
    
                    return df_past.join(df_future, how='left')
                
                # Generate forecasts for the future
                df_forecast = multi_step_forecasts_future(n_past=0, n_future=forecast_window_val)
                df_forecast = df_forecast.reset_index()
                df_forecast['part_number'] = product
                df_forecast['last_history'] = (pd.to_datetime(training_end_date_val) + relativedelta(months=t)).strftime('%Y-%m')
                
                # Append forecasts to the corresponding lag data
                if t == 0:
                    fin_data_1 = pd.concat([fin_data_1, df_forecast])
        
            except Exception as e:
                print(f'Error for product {product}: {str(e)}')
                df_forecast = pd.DataFrame({
                    'period': [np.nan],
                    'part_number': [product],
                    'Actual': [np.nan],
                    'Forecast': [np.nan],
                    'last_history': [np.nan]
                })
                if t == 0:
                    fin_data_1 = pd.concat([fin_data_1, df_forecast])
                elif t == 1:
                    fin_data_2 = pd.concat([fin_data_2, df_forecast])
                elif t == 2:
                    fin_data_3 = pd.concat([fin_data_3, df_forecast])
    
        # Append data for each product to the overall outputs
        out_data_1 = pd.concat([out_data_1, fin_data_1])
    return out_data_1

In [14]:
print(datetime.now())

2024-09-15 09:32:07.572185


In [15]:
training_date = pd.to_datetime(config.get_value('model_training_date'))
file_date = training_date.strftime("%Y-%m-%d")

In [16]:
import warnings
warnings.filterwarnings('ignore')

training_end_date = (pd.to_datetime(data_new_1['period']).max() + relativedelta(months=-3)).strftime('%Y-%m-%d')
parts_list_updated = parts_list[6000:]  # Line to be commented post testing
# parts_list_updated = parts_list[:]    # Line to be uncommented post testing
lstm_training_data = create_model_output(data_new_1, training_end_date, n_input, n_output, bucket, prefix_lstm_pickle, 3, parts_list_updated)

Running for product number 1


2024-09-15 09:32:08.661505: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Running for product number 2


Running for product number 3


Running for product number 4


...

Running for product number 518


Running for product number 519


Running for product number 520


Running for product number 521


Running for product number 522
Error for product PS382BY004: Input 0 of layer "lstm_1042" is incompatible with the layer: expected ndim=3, found ndim=1. Full shape received: (None,)
Running for product number 523
Error for product PC55712001: Input 0 of layer "lstm_1043" is incompatible with the layer: expected ndim=3, found ndim=1. Full shape received: (None,)
Running for product number 524
Error for product 2880037111: Input 0 of layer "lstm_1044" is incompatible with the layer: expected ndim=3, found ndim=1. Full shape received: (None,)
Running for product number 525
Error for product PC1860K1Q3: Input 0 of layer "lstm_1045" is incompatible with the layer: expected ndim=3, found ndim=1. Full shape received: (None,)


In [17]:
print(datetime.now())

2024-09-15 10:20:37.796378


In [18]:
datetime.now().strftime("%Y-%m-%d")

'2024-09-15'

In [19]:
prefix_output

'persist-bo/persist-bo-demand-forecast/tpcap-models/20240913/model-training-output'

In [20]:
lstm_1 = get_latest_s3_file(bucket, prefix_output+"/", f'1_{config.get_value("file_name_training_lstm")}1_')
lstm_2 = get_latest_s3_file(bucket, prefix_output+"/", f'2_{config.get_value("file_name_training_lstm")}2_')
lstm_3 = get_latest_s3_file(bucket, prefix_output+"/", f'3_{config.get_value("file_name_training_lstm")}3_')
lstm_4 = get_latest_s3_file(bucket, prefix_output+"/", f'4_{config.get_value("file_name_training_lstm")}4_')
lstm_5 = get_latest_s3_file(bucket, prefix_output+"/", f'5_{config.get_value("file_name_training_lstm")}5_')
lstm_6 = get_latest_s3_file(bucket, prefix_output+"/", f'6_{config.get_value("file_name_training_lstm")}6_')

In [21]:
total_lstm_data = pd.concat([
    pd.read_excel(f"s3://{bucket}/"+lstm_1),
    pd.read_excel(f"s3://{bucket}/"+lstm_2),
    pd.read_excel(f"s3://{bucket}/"+lstm_3),
    pd.read_excel(f"s3://{bucket}/"+lstm_4),
    pd.read_excel(f"s3://{bucket}/"+lstm_5),
    pd.read_excel(f"s3://{bucket}/"+lstm_6),
    lstm_training_data
], ignore_index=True)

In [22]:
total_lstm_data['part_number'].nunique()

6525

In [23]:
# total_lstm_data[total_lstm_data['part_number']=='PC1860K1Q3']

In [24]:
file_name_training_lstm = f'{config.get_value("file_name_training_lstm")}{file_date}.xlsx'
file_path_training_lstm = f's3://{bucket}/{prefix_output}/{file_name_training_lstm}'

total_lstm_data.to_excel(file_path_training_lstm, index=False)

In [25]:
end_time = datetime.now()
print("Total time taken for LSTM Training 7::", (end_time-start_time).seconds/60, " minutes")

Total time taken for LSTM Training 7:: 50.11666666666667  minutes
